## Quickstart: Getting Started with the Snowflake Python API

To start, import the required libraries. We'll import Snowpark, the core Snowflake Python API library, and several classes within the core library that are used to manage Snowflake objects. 

In [ ]:
from snowflake.snowpark import Session
from snowflake.core import Root
from snowflake.core.database import Database
from snowflake.core.schema import Schema
from snowflake.core.table import Table, TableColumn, PrimaryKey
from snowflake.core.warehouse import Warehouse

Next, configure the `connection_params` dictionary below with your Snowflake account credentials. The account identifier, username, and password are required parameters. Optionally, you can specify a database, schema, and warehouse. For more information on the various connection methods, see [Connecting to Snowflake with the Snowflake Python API](https://docs.snowflake.com/en/LIMITEDACCESS/snowflake-python-api/snowflake-python-connecting-snowflake).

In [ ]:
connection_params = {
    "account": "ACCOUNT-IDENTIFER",
    "user": "USERNAME",
    "password": "PASSWORD"
}

Next, create your Snowpark session by passing in your connection parameters.

In [ ]:
session = Session.builder.configs(connection_params).create()

Next, instantiate the `Root` class and pass in your session object as an argument. The resulting object – in this case, `root` – will be used to invoke the rest of Snowflake Python API methods and types.

In [ ]:
root = Root(session)

Let's start by using `root`  to create a database, schema, and table.

In [ ]:
database = root.databases.create(Database(name="PYTHON_API_DB"), mode="orreplace")

If you'd like, you can momentarily navigate to your Snowflake account in the browser and verify that the database object `PYTHON_API_DB` was created. You can do this for the next two cells as well.

In [ ]:
schema = database.schemas.create(Schema(name="PYTHON_API_SCHEMA"), mode="orreplace")

In [ ]:
table = schema.tables.create(Table(
        name="PYTHON_API_TABLE", 
        columns=[TableColumn("TEMPERATURE", "int", nullable=False), TableColumn("LOCATION", "string")],
    ), 
    mode="orreplace")

You should now have a database, schema, and table within your Snowflake account. 

`table` represents a `TableResource` object. To retrieve information about the table, call `.fetch()` on the table you created above and store it an object. 

In [ ]:
table_details = table.fetch()

`.fetch()` returns a `TableModel` object, and objects of this type have several methods associated with them that can return more information about that specific Snowflake object. Let's call `.to_dict()` on `python_api_table` to view this in action.

In [ ]:
table_details.to_dict()

As you can see, the dictionary above contains information about the `PYTHON_API_TABLE` table that you created earlier.

Notice that information, like the columns in the table, for example, were not included in the dictionary. To include this information, pass in the `deep` argument to `fetch()` and set it to `True`.

In [ ]:
table_details_full = table.fetch(deep=True)

In [ ]:
table_details_full.to_dict()

Note that column information is now included in the dictionary.

Let's now add a new column to the table.

In [ ]:
table_details_full.columns.append(TableColumn(name="ELEVATION", datatype="int", nullable=False, constraints=[PrimaryKey()]))

In the cell above, we define a new column using `TableColumn` and add it to the array in the TableModel that represents the column in the table. To add the column to the table, run the next cell.

In [ ]:
table.create_or_update(table_details_full)

Navigate to your Snowflake account and confirm that the new column was added to the table.

In [ ]:
table.fetch(deep=True).to_dict()

Now let's create a warehouse.

In [ ]:
warehouses = root.warehouses

The cell above returns the collection of warehouses associated with your session. Let's define a new warehouse.

In [ ]:
python_api_wh = Warehouse(
    name="PYTHON_API_WH",
    warehouse_size="SMALL",
    auto_suspend=500,
)

warehouse = warehouses.create(python_api_wh)

The cell above defines a new warehouse and creates it in your Snowflake account. Navigate to your account to confirm that the warehouse was created.

Now let's grab some information related to the warehouse.

In [ ]:
warehouse_details = warehouse.fetch()
warehouse_details.to_dict()

We can also iterate through and search the warehouses in your Snowflake account.

In [ ]:
warehouse_list = warehouses.iter(like="PYTHON_API_WH")
result = next(warehouse_list)
result.to_dict()

In the cell above, we search for all warehouses that have a name that matches `PYTHON_API_WH`. In this case, we passed in the exact name of the warehouse we created earlier, but the `like` argument is generally a case-insensitive string that functions as a filter, with support for SQL wildcard characters like `%` and `_`.

Let's now programmatically change the size of the warehouse.

In [ ]:
warehouse.create_or_update(Warehouse(
    name="PYTHON_API_WH",
    warehouse_size="LARGE",
    auto_suspend=500,
))

Confirm that the warehouse size was updated. You can also navigate to your Snowflake account to confirm the change in warehouse esize.

In [ ]:
warehouse.fetch().size

Finally, delete the warehouse and close your Snowflake connection.

In [ ]:
warehouse.delete()
session.close()

Congratulations! In this Quickstart, you learned the fundamentals for managing Snowflake objects using the Snowflake Python API. Bookmark this Quickstart and be sure to come back soon, when the Quickstart will be expanded to include Task, DAG, and Snowpark Container Services management using the Snowflake Python API.

For more information, see the following resources:

* [Snowflake Documentation: Snowflake Python API](https://docs.snowflake.com/en/LIMITEDACCESS/snowflake-python-api/snowflake-python-overview)

* [Snowflake Python API Reference Documentation](https://docs.snowflake.com/en/LIMITEDACCESS/snowflake-python-api/reference/0.1.0/index.html)